In [2]:
import sys
import argparse
import random
import openslide
import PIL.Image as Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.models as models
import utility_functions as uf
import pandas as pd
import os
import shutil
import xlwt
from xlwt import Workbook
import xlrd

In [15]:
#checking the data
lib = torch.load("/home/weber50432/AML_image_processing/lib/NPM1_patch_test/NPM1_patch_test_train_data.pt")
print(lib["slides"])
print(lib["targets"])
print(lib["grid"][:2][:5])
grid = []
slideIDX = []
count = 0
for i,g in enumerate(lib['grid']):
  grid.extend(g)
  slideIDX.extend([i]*len(g))
print(len(grid))
print(len(slideIDX))

['A3.ndpi', 'A9.ndpi', 'A12.ndpi', 'A13.ndpi', 'A26.ndpi', 'A27.ndpi', 'A28.ndpi', 'A29.ndpi', 'A60.ndpi', 'A99.ndpi', 'A102.ndpi', 'A104.ndpi', 'A219.ndpi', 'A1001.ndpi']
[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1]
[['/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_32996.png', '/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_53204.png', '/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_29422.png', '/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_16468.png', '/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_25430.png', '/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_51398.png', '/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_27968.png', '/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_30844.png', '/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_13148.png', '/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_28322.png', '/home/exon_storage1/aml_slide/patches/A3/rightside_p

In [4]:
# test the get_grid function
y = uf.get_grid("/home/exon_storage1/aml_slide/",[413],256,0.6,0.15,500)
print(len(y[0]))
print(y[0][:10])

slide 413 done, slect 500 patches from 1356 total patches.
500
[(97280, 26368), (110080, 77056), (94720, 31232), (98048, 79104), (112128, 69888), (118272, 31744), (112640, 72192), (95744, 23040), (113152, 73216), (95488, 35072)]


In [3]:
# test the get_patches_grid function
path = "/home/weber50432/AML_image_processing/HCT_region_select/output/"
slide_list = [3, 9, 12, 13,26, 27, 28, 29,59, 60, 99, 102, 103, 104, 105, 106, 108,109,153,219,349,1001]
grid = uf.get_patches_grid(path,slide_list,1)
print(grid)

slide A59 do not have excel file.
[['/home/exon_storage1/aml_slide/patches/A3/rightside_patch/A3_29394.png'], ['/home/exon_storage1/aml_slide/patches/A9/rightside_patch//A9_30406.png'], ['/home/exon_storage1/aml_slide/patches/A12/rightside_patch/A12_15986.png'], ['/home/exon_storage1/aml_slide/patches/A13/rightside_patch/A13_28565.png'], ['/home/exon_storage1/aml_slide/patches/A26/rightside_patch/A26_29415.png'], ['/home/exon_storage1/aml_slide/patches/A27/rightside_patch/A27_40321.png'], ['/home/exon_storage1/aml_slide/patches/A28/rightside_patch/A28_06153.png'], ['/home/exon_storage1/aml_slide/patches/A29/rightside_patch/A29_29625.png'], ['/home/exon_storage1/aml_slide/patches/A60/rightside_patch/A60_30633.png'], ['/home/exon_storage1/aml_slide/patches/A99/rightside_patch/A99_32389.png'], ['/home/exon_storage1/aml_slide/patches/A102/rightside_patch/A102_63064.png'], ['/home/exon_storage1/aml_slide/patches/A103/rightside_patch/A103_34084.png'], ['/home/exon_storage1/aml_slide/patches/

In [11]:
# replace the // with / in excel file in the output folder
excel_path = "/home/weber50432/AML_image_processing/HCT_region_select/output/A13/porduction_result.xls"
if os.path.exists(excel_path):
  wb = xlrd.open_workbook(excel_path)
  sheet1 = wb.sheet_by_index(0)
  sheet2 = wb.sheet_by_index(1)
  pos_list = sheet1.col_values(0)
  neg_list = sheet2.col_values(0)
  pos_list.pop(0)
  neg_list.pop(0)
  wb = Workbook()
  sheetNumber0 = wb.add_sheet('Sheet 0')
  sheetNumber1 = wb.add_sheet('Sheet 1')

  row = 0
  sheetNumber0.write(row, 0, 'Posivitve List')
  sheetNumber0.write(row, 1, len(pos_list))
  row += 1
  for i in pos_list:
    new_string = i.replace("//", "/")
    sheetNumber0.write(row, 0, new_string)
    row += 1

  row = 0
  sheetNumber1.write(row, 0, 'Negative List')
  sheetNumber1.write(row, 1, len(neg_list))
  row += 1
  for i in neg_list:
    new_string = i.replace("//", "/")
    sheetNumber1.write(row, 0, new_string)
    row += 1
  wb.save(excel_path)